# [Module 2.1] 훈련 아테펙트를 Re-Packaging 하기

이 노트북은 아래와 같은 작업을 합니다. 

---



In [1]:
%store -r artifact_path

---
# 1. 패키징 모델
- 패키징할 폴더 생성
- 훈련된 모델 아티펙트 다운로드 및 해제
- 모델 아티펙트, 추론 코드의 폴더 지정 후에 패키징 폴더에 복사
- 패키징 폴더를 model.tar.gz 압축 후에 S3에 업로딩
- 



In [2]:
import os

package_dir = 'model_pkg'
os.makedirs(package_dir, exist_ok=True)



추론 코드가 있는 폴더

In [3]:
code_dir = '../../../src/*'

In [4]:
%%sh -s {artifact_path} {package_dir} {code_dir}

artifact_path=$1
package_dir=$2
code_dir=$3

cd $package_dir # 폴더 생성
echo $PWD
rm -rf ./*
aws s3 cp $artifact_path . # 아티펙트 다운로드
tar -xzvf model.tar.gz # 압축 해제
rm model.tar.gz # 압축 파일 삭제
mkdir -p code # code 폴더 생성
cp -r $code_dir code/ # src 파일 모두 카피
rm -rf code/__pycache__
tar -czvf model.tar.gz * # model.tar.gz 파일 생성

/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/sagemaker/recommendation/Neural-Collaborative-Filtering-On-SageMaker/2_Inference/sagemaker_inference_container/container-inference/model_pkg
download: s3://sagemaker-us-east-1-057716757052/pytorch-training-2023-10-28-07-43-01-333/output/model.tar.gz to ./model.tar.gz


tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'


NeuMF-end.pth
code/
code/benchmark_util.py
code/inference_utils.py
code/requirements.txt
code/common_utils.py
code/inference.py
code/config.py
code/model_config.json
code/model.py
code/data_utils.py
NeuMF-end.pth


# 2. S3 에 업로드

In [5]:
import sagemaker
sagemaker_session  = sagemaker.session.Session()
prefix='ncf/inference'
bucket = sagemaker.session.Session().default_bucket()
print("bucket: ", bucket)

model_path = os.path.join(package_dir, 'model.tar.gz')
byom_artifact = sagemaker_session.upload_data(model_path, bucket, prefix)
print("byom_artifact: \n", byom_artifact)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
bucket:  sagemaker-us-east-1-057716757052
byom_artifact: 
 s3://sagemaker-us-east-1-057716757052/ncf/inference/model.tar.gz


In [6]:
%store byom_artifact

Stored 'byom_artifact' (str)
